In [1]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.2.0
    Uninstalling fsspec-2024.2.0:
      Successfully uninstalled fsspec-2024.2.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2023.12.2.post1 requires fsspec==2023.12.2, but you have fsspec 2023.10.0 which is incompatible.
s3fs 2024.2.0 requires fsspec==2024.2.0, but you have fsspec 2023.10.0 which is incompatible.


# Sources:
1 - https://karpathy.ai/zero-to-hero.html  
2 - https://www.youtube.com/watch?v=kCc8FmEb1nY  
3 - https://github.com/karpathy/minbpe/tree/master  

# Imports

In [2]:
import regex as re
from datasets import load_dataset
from collections import Counter, OrderedDict
from tqdm import trange
import json
from tqdm import tqdm

# Parameters

In [3]:
# Dataset parameters
train_ratio = 0.9
sample_size = 1_000

# Parameters for Additional test
max_single_char_vocab_size = 150
max_vocab_size = 500
min_freq = 100
UNK_TOKEN = "<UNK>"

# select a tokenizer among BasicTokenizer, RegexTokenizer and MostCommonRegexTokenizer
tokenizer_name = "MostCommonRegexTokenizer"

SEED = 15

# Load files

In [4]:
# Define the path to the dataset
dataset_name = "20231101.fr"

# Load the dataset
raw_dataset = load_dataset("wikimedia/wikipedia", dataset_name)
print(f"Dataset size: {raw_dataset['train'].num_rows}")

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2564646 [00:00<?, ? examples/s]

Dataset size: 2564646


# Data preparation

In [5]:
# Create training and evaluation datasets
if sample_size < 0:
    train_sample = round(raw_dataset['train'].num_rows * train_ratio)
    test_sample = round(raw_dataset['train'].num_rows * (1- train_ratio))
else:
    train_sample = round(sample_size * train_ratio)
    test_sample = round(sample_size * (1- train_ratio))

ds_train_test = raw_dataset['train'].train_test_split(train_size=train_sample, test_size=test_sample, seed=SEED)
train_text = ''.join([t['text'] for t in ds_train_test['train']])

In [6]:
# Get some statistics about the data
print("Number of unique words: ", len(set(train_text.split())))
print("Number of unique characters: ", len(set(train_text)))
counter = Counter(train_text)
chars_above_threshold = [c for c in counter if counter[c] >= min_freq]
print("Number of unique characters above threshold: ", len(chars_above_threshold))
chars_below_threshold = [c for c in counter if counter[c] < min_freq]
print("Number of unique characters below threshold: ", len(chars_below_threshold))

Number of unique words:  74972
Number of unique characters:  874
Number of unique characters above threshold:  110
Number of unique characters below threshold:  764


# Step 1
Create a simple Byte Pair Encoder (BPE)

In [7]:
class BasicTokenizer():
    def __init__(self):
        self.text_to_int = {}  # {text: int}
        self.int_to_text = {}  # {int: text}
        self.merged_dict = OrderedDict()  # {(token, token): token}
        
    def train(self, text, max_vocab_size, verbose=False):
        self.text_to_int, self.int_to_text = self._create_initial_vocabulary(text)  
        self.current_vocab_size = len(self.text_to_int)
        
        train_tokens = self.encode(text) 
        
        if verbose: 
            self.print_stats(train_tokens)
        
        if self.current_vocab_size < max_vocab_size:
            iter_count = max_vocab_size - self.current_vocab_size
            for _ in trange(iter_count):
                token_pairs = self._get_pairs(train_tokens)
                most_freq = self._get_most_common_token_pair(token_pairs)
                
                str_pair = ''.join([self.int_to_text[most_freq[0]], self.int_to_text[most_freq[1]]])
                self.text_to_int[str_pair] = self.current_vocab_size
                self.int_to_text[self.current_vocab_size] = str_pair
                self.merged_dict[most_freq] = self.text_to_int[str_pair]
                self.current_vocab_size += 1

                new_train_tokens = self._replace_token_pairs(train_tokens, most_freq, self.text_to_int[str_pair])
                train_tokens = new_train_tokens

                if verbose:
                    self.print_stats(train_tokens, str_pair, most_freq)

    def print_stats(self, train_tokens, str_pair="", token_pair=None):
        print("Updated vocab size:", self.current_vocab_size, 
              "Updated sequence length:", len(train_tokens), 
              "New vocab:", str_pair, 
              "Token pair:", token_pair)
        
    def _get_pairs(self, tokens):
        return list(zip(tokens, tokens[1:]))
    
    def _create_initial_vocabulary(self, text):
        sorted_text = sorted(set(text))
        text_to_int = {c: i for i, c in enumerate(sorted_text)}
        int_to_text = {i: c for i, c in enumerate(sorted_text)}
        return text_to_int, int_to_text

    def _get_most_common_token_pair(self, token_pairs):
        counter = Counter(token_pairs)
        most_freq = counter.most_common(1)[0][0]
        return most_freq
    
    def _replace_token_pairs(self, tokens, target_pair, new_token):
        """
        Replace all occurrences of `target_pair` in `tokens` by `new_token`.
        """
        new_tokens = []
        i = 0
        while i < len(tokens) - 1:
            if (tokens[i], tokens[i+1]) == target_pair:
                new_tokens.append(new_token)
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1
        if i < len(tokens):
            new_tokens.append(tokens[i])
        return new_tokens

    def encode(self, text):
        tokens = [self.text_to_int[char] for char in text]
        for key in self.merged_dict:
            tokens = self._replace_token_pairs(tokens, key, self.merged_dict[key])
        return tokens
            
    
    def decode(self, tokens):
        return "".join([self.int_to_text[i] for i in tokens])


In [8]:
if tokenizer_name == "BasicTokenizer":
    tokenizer = BasicTokenizer()
    tokenizer.train(train_text, max_vocab_size=max_vocab_size, verbose=False)

    text = "Bonjour le monde, comment va ta journée?"
    code = tokenizer.encode(text)
    print(f"len original text: {len(text)}, len bpe text: {len(code)}") 
    print(code)
    print(tokenizer.decode(code))

# Step 2
Improve the BPE from step 1 by splitting first the text into words and then applying the BPE to each word. This will avoid having tokens that are composed of multiple words or heterogeneous (word + number for example)

In [9]:
class RegexTokenizer():
    def __init__(self):
        self.text_to_int = {}  # {text: int}
        self.int_to_text = {}  # {int: text}
        self.merged_dict = OrderedDict()  # {(token, token): token}
        
    def train(self, text, max_vocab_size, verbose=False, show_progress=False):
        self.text_to_int, self.int_to_text = self._create_initial_vocabulary(text)
        self.current_vocab_size = len(self.text_to_int)
        
        tokens_splits = self.encode(text)
        
        if verbose: 
            self.print_stats(tokens_splits)
            
        if self.current_vocab_size < max_vocab_size:
            iter_count = max_vocab_size - self.current_vocab_size
            for _ in trange(iter_count, disable=not show_progress):
                token_pairs = []
                for tokens_split in tokens_splits:
                    if len(tokens_split) > 1:
                        token_pairs.extend(self._get_pairs(tokens_split))
                        
                most_freq = self._get_most_common_token_pair(token_pairs)
                
                str_pair = ''.join([self.int_to_text[most_freq[0]], self.int_to_text[most_freq[1]]])
                self.text_to_int[str_pair] = self.current_vocab_size
                self.int_to_text[self.current_vocab_size] = str_pair
                self.merged_dict[most_freq] = self.text_to_int[str_pair]
                
                new_tokens_splits = [self._replace_token_pairs(t, most_freq, self.current_vocab_size) for t in tokens_splits]
                tokens_splits = new_tokens_splits
                self.current_vocab_size += 1
                
                if verbose: 
                    self.print_stats(tokens_splits, str_pair, most_freq)

    def print_stats(self, tokens_splits, str_pair="", token_pair=None):
        print("Updated vocab size:", self.current_vocab_size, 
              "Updated sequence length:", sum([len(split) for split in tokens_splits]),
              "New vocab:", str_pair, 
              "Token pair:", token_pair)
        
    def _get_pairs(self, tokens):
        return list(zip(tokens, tokens[1:]))
    
    def _create_initial_vocabulary(self, text):
        sorted_text = sorted(set(text))
        text_to_int = {c: i for i, c in enumerate(sorted_text)}
        int_to_text = {i: c for i, c in enumerate(sorted_text)}
        return text_to_int, int_to_text

    def _get_most_common_token_pair(self, token_pairs):
        counter = Counter(token_pairs)
        most_freq = counter.most_common(1)[0][0]
        return most_freq
    
    def _replace_token_pairs(self, tokens, target_pair, new_token):
        """
        Replace all occurrences of `target_pair` in `tokens` by `new_token`.
        """
        new_tokens = []
        i = 0
        while i < len(tokens) - 1:
            if (tokens[i], tokens[i+1]) == target_pair:
                new_tokens.append(new_token)
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1
        if i < len(tokens):
            new_tokens.append(tokens[i])
        return new_tokens

    def _simple_encode(self, text):
        
        tokens = [self.text_to_int[char] for char in text]
        for key in self.merged_dict:
            tokens = self._replace_token_pairs(tokens, key, self.merged_dict[key])
        return tokens
    
    def encode(self, text):
        text_splits = self._split_text(text)
        tokens_splits = [self._simple_encode(text_split) for text_split in text_splits]
        return tokens_splits
        
    def _split_text(self, text):
        split_text = re.findall(GPT4_SPLIT_PATTERN, text)
        return split_text
    
    def decode(self, tokens_splits):
        texts = []
        for tokens in tokens_splits:
            texts.append("".join([self.int_to_text[i] for i in tokens]))
        return "".join(texts)
    
    def save_vocab(self, path):
        int_to_text = self.int_to_text
        with open(path, "w") as f:
            json.dump(int_to_text, f)


In [10]:
if tokenizer_name == "RegexTokenizer":
    GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
    tokenizer = RegexTokenizer()
    tokenizer.train(train_text, max_vocab_size=max_vocab_size, verbose=True, show_progress=True)
    tokenizer.save_vocab("RegexTokenizerVocab.txt")

    text = "Bonjour le monde, comment va ta journée?"
    code = tokenizer.encode(text)
    print(f"len original text: {len(text)}, len bpe text: {sum([len(c) for c in code])}")
    print(code)
    print(tokenizer.decode(code))

In [11]:
if tokenizer_name == "RegexTokenizer":
    tokenized_text = tokenizer.encode(train_text)
    flat_tokenized_text = [item for sublist in tokenized_text for item in tqdm(sublist)]  # flatten code
    counter = Counter(flat_tokenized_text)
    counter

In [12]:
if tokenizer_name == "RegexTokenizer":
    print([(v, tokenizer.int_to_text[v], count) for v, count in counter.most_common()])

# Additional test
Remove least common characters from the dictionary (below a threshold value) before merging characters  
This step will help to have a much smaller dictionary and will speed up the process  
Need to create a fallback in case of no common characters is found during the encoding

In [13]:
class MostCommonRegexTokenizer():
    def __init__(self):
        self.text_to_int = {UNK_TOKEN: 0}  # {text: int}
        self.int_to_text = {0: UNK_TOKEN}  # {int: text}
        self.merged_dict = OrderedDict()  # {(token, token): token}
        
    def train(self, text, max_vocab_size, min_freq, max_single_char_vocab_size, verbose=False, show_progress=False):
        self.text_size = len(text)
        self.char_counter = Counter(text)
        filtered_text = self.remove_least_common_characters(text, min_freq)
        text_to_int, int_to_text = self._create_initial_vocabulary(filtered_text, max_single_char_vocab_size)
        self.text_to_int.update(text_to_int)
        self.int_to_text.update(int_to_text)
        self.current_vocab_size = len(self.text_to_int)
        
        tokens_splits = self.encode(filtered_text)
        
        if verbose: 
            self._print_stats(tokens_splits)
        
        print('Creating merged dictionary...')
        if self.current_vocab_size < max_vocab_size:
            iter_count = max_vocab_size - self.current_vocab_size
            for _ in trange(iter_count, disable=not show_progress):
                token_pairs = []
                for tokens_split in tokens_splits:
                    if len(tokens_split) > 1:
                        token_pairs.extend(self._get_pairs(tokens_split))
                        
                most_freq = self._get_most_common_token_pair(token_pairs)
                
                str_pair = ''.join([self.int_to_text[most_freq[0]], self.int_to_text[most_freq[1]]])
                self.text_to_int[str_pair] = self.current_vocab_size
                self.int_to_text[self.current_vocab_size] = str_pair
                self.merged_dict[most_freq] = self.text_to_int[str_pair]
                
                new_tokens_splits = [self._replace_token_pairs(t, most_freq, self.current_vocab_size) for t in tokens_splits]
                tokens_splits = new_tokens_splits
                self.current_vocab_size += 1
                
                if verbose: 
                    self._print_stats(tokens_splits, str_pair, most_freq)

    def _print_stats(self, tokens_splits, str_pair="", token_pair=None):
        current_text_size = sum([len(split) for split in tokens_splits])
        print(f"Updated vocab size: {self.current_vocab_size} "
              f"Updated sequence length: {current_text_size} "
              f"New vocab: {repr(str_pair)} "
              f"Token pair: {token_pair} "
              f"Compression ratio: {self.text_size / current_text_size - 1:.2f}% "
             )
        
    def _get_pairs(self, tokens):
        """ Returns a list of tuples containing pairs of adjacent tokens from the input tokens list"""
        return list(zip(tokens, tokens[1:]))
    
    def _create_initial_vocabulary(self, text, max_single_char_vocab_size):        
        print("Creating initial vocabulary...")
        sorted_char = sorted(set(text))
        most_common_chars = [char for char, count in self.char_counter.most_common(max_single_char_vocab_size)]
        filtered_sorted_char = [c for c in sorted_char if c in most_common_chars]
        start_id = len(self.text_to_int)
        text_to_int = {c: start_id+i for i, c in enumerate(filtered_sorted_char)}
        int_to_text = {start_id+i: c for i, c in enumerate(filtered_sorted_char)}
        return text_to_int, int_to_text

    def _get_most_common_token_pair(self, token_pairs):
        """Gets the most frequent token pair from the given list of token pairs.
    
        This ignores token pairs that contain <UNK> tokens. It will raise a ValueError if no
        valid token pairs are found.
        
        Args:
            token_pairs: List of token id pairs
        
        Returns:
            The most frequent token id pair
        """
        counter = Counter(token_pairs)
        # Get the first most frequent pair (excluding <UNK>)
        for pair, freq in counter.most_common():
            if pair[0]!= 0 and pair[1]!= 0:
                return pair
        else:
            raise ValueError("No pairs found")
    

    def _replace_token_pairs(self, tokens, target_pair, new_token):
        """
        Replaces occurrences of a given token pair in the input 
        token list with a new token.
        
        Iterates through tokens checking for the target pair. 
        When found, appends the new token to the output and 
        advances the index by 2 to skip the pair. Otherwise just 
        appends the current token.
        
        Returns the new list of tokens.
        """
        new_tokens = []
        i = 0
        while i < len(tokens) - 1:
            if (tokens[i], tokens[i+1]) == target_pair:
                new_tokens.append(new_token)
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1
        if i < len(tokens):
            new_tokens.append(tokens[i])
        return new_tokens


    def _simple_encode(self, text):
        """Encodes the given text into a list of token IDs.
    
        This uses a simple encoding scheme of looking up each character in the 
        text_to_int mapping and falling back to the unknown token ID if not found.
        
        It then iterates through the tokens, looking for any token pairs that match
        the merged_dict and replacing them with the merged token ID.
        
        Args:
            text: The text to encode
        
        Returns: 
            The list of encoded token IDs.
        """
        tokens = [self.text_to_int.get(char, 0) for char in text]  # fallback to <UNK> when not found
        for key in self.merged_dict:
            tokens = self._replace_token_pairs(tokens, key, self.merged_dict[key])
        return tokens
    
    def encode(self, text):
        print("Encoding...")
        text_splits = self._split_text(text)
        tokens_splits = [self._simple_encode(text_split) for text_split in tqdm(text_splits)]
        return tokens_splits
        
    def _split_text(self, text):
        split_text = re.findall(GPT4_SPLIT_PATTERN, text)
        return split_text
    
    def decode(self, tokens_splits):
        print("Decoding...")
        texts = []
        for tokens in tokens_splits:
            texts.append("".join([self.int_to_text[i] for i in tokens]))
        return "".join(texts)
    
    def save_vocab(self, path):
        print("Saving vocab...")
        int_to_text = self.int_to_text
        with open(path, "w") as f:
            json.dump(int_to_text, f)
                
    def remove_least_common_characters(self, text, min_freq):
        """
        Remove characters that appear less than `min_freq` in `text` using regex.
        """
        print("Removing least common characters...")
        least_common_chars = {c for c in self.char_counter if self.char_counter[c] < min_freq}
        if least_common_chars:  
            pattern = '[' + re.escape(''.join(least_common_chars)) + ']'
            filtered_text = re.sub(pattern, '', text)
            return filtered_text
        else:
            return text


In [14]:
%%time
if tokenizer_name == "MostCommonRegexTokenizer":
    GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
    tokenizer = MostCommonRegexTokenizer()
    tokenizer.train(
        text=train_text, 
        max_vocab_size=max_vocab_size, 
        min_freq=min_freq, 
        max_single_char_vocab_size=max_single_char_vocab_size,
        verbose=True, 
        show_progress=True)
    tokenizer.save_vocab("MostCommonRegexTokenizer.txt")

    text = "Bonjour le monde, comment va ta journée?"
    code = tokenizer.encode(text)
    print(f"len original text: {len(text)}, len bpe text: {sum([len(c) for c in code])}")
    print(code)
    print(tokenizer.decode(code))

Removing least common characters...
Creating initial vocabulary...
Encoding...


100%|██████████| 556316/556316 [00:01<00:00, 304702.66it/s]


Updated vocab size: 111 Updated sequence length: 2720406 New vocab: '' Token pair: None Compression ratio: 0.00% 
Creating merged dictionary...


  0%|          | 1/389 [00:02<18:05,  2.80s/it]

Updated vocab size: 112 Updated sequence length: 2662698 New vocab: ' d' Token pair: (2, 58) Compression ratio: 0.02% 


...

Updated vocab size: 499 Updated sequence length: 1314917 New vocab: 'ét' Token pair: (94, 74) Compression ratio: 1.07% 


100%|██████████| 389/389 [11:53<00:00,  1.83s/it]


Updated vocab size: 500 Updated sequence length: 1314314 New vocab: 'ass' Token pair: (189, 73) Compression ratio: 1.07% 
Saving vocab...
Encoding...


100%|██████████| 9/9 [00:00<00:00, 1505.73it/s]

len original text: 40, len bpe text: 20
[[28, 115, 64, 163], [144], [141, 115, 207], [10], [210, 168], [184, 55], [154, 55], [243, 163, 68, 200], [0]]
Decoding...
Bonjour le monde, comment va ta journée<UNK>
CPU times: user 11min 53s, sys: 4.02 s, total: 11min 57s
Wall time: 11min 57s


In [15]:
if tokenizer_name == "MostCommonRegexTokenizer":
    counter = Counter(train_text)
    counter.most_common()

In [16]:
if tokenizer_name == "MostCommonRegexTokenizer":
    tokenized_text = tokenizer.encode(train_text)
    flat_tokenized_text = [item for sublist in tokenized_text for item in sublist]  # flatten code
    counter = Counter(flat_tokenized_text)
    print('\n'.join(f'key:{key:<8} str:{repr(tokenizer.int_to_text[key]):<10} count:{value}' for key, value in counter.most_common()))

Encoding...


100%|██████████| 557149/557149 [04:31<00:00, 2050.43it/s]


key:2        str:' '        count:33220
key:117      str:' de'      count:26341
key:10       str:','        count:25110
key:73       str:'s'        count:18508
key:55       str:'a'        count:16118
key:59       str:'e'        count:16074
key:112      str:'es'       count:14013
key:57       str:'c'        count:13260
key:63       str:'i'        count:13103
key:122      str:'er'       count:12889
key:74       str:'t'        count:12607
key:69       str:'o'        count:12014
key:134      str:' la'      count:11810
key:70       str:'p'        count:11781
key:12       str:'.'        count:11714
key:94       str:'é'        count:11611
key:113      str:' l'       count:11416
key:66       str:'l'        count:10687
key:115      str:'on'       count:10593
key:7        str:"'"        count:10443
key:75       str:'u'        count:10426
key:139      str:' et'      count:10232
key:121      str:' s'       count:10229
key:111      str:' d'       count:10169
key:61       str:'g'        count:9843
k